In [1]:
import os, json, re, gc
from pathlib import Path
from PIL import Image
import torch, numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForVision2Seq


2025-05-06 06:01:50.283907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746511310.458723      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746511310.512535      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
MODEL_ID       = "llava-hf/llava-1.5-13b-hf"
USE_FP16       = True
BATCH_SIZE     = 4
MAX_NEW_TOKENS = 80
DEVICE         = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BASE_DIR       = Path("/kaggle/input")
FP_ATTR_CSV    = BASE_DIR/"nlp-fashionpedia"/"predicted_attributes_final.csv"
FP_IMG_DIR     = BASE_DIR/"nlp-fashionpedia"/"train_sample"/"train_sample"
FP_LABELS_JSON = BASE_DIR/"nlp-fashionpedia"/"label_descriptions.json"
INDO_META_CSV  = BASE_DIR/"indofashion-nlp"/"indofashion_metadata.csv"
INDO_IMG_DIR   = BASE_DIR/"indofashion-nlp"/"train_sand"/"train_sand"
OUTPUT_CSV     = Path("UnifiedDescriptions.csv")
CKPT_JSON      = Path("llava_unified_checkpoint.json")

In [3]:
def load_label_map(p):
    d = json.load(open(p))
    return {a["id"]: a["name"] for a in d["attributes"]}

attr_id_to_name = load_label_map(FP_LABELS_JSON)

def build_fp_prompt(attr_str):
    # Only evaluate if string looks like "[...]", else skip
    if not isinstance(attr_str, str) or not attr_str.strip().startswith("["):
        return ""
    vec = eval(attr_str, {"np":np, "int32":np.int32})
    names = [attr_id_to_name[i] for i,v in enumerate(vec) if v==1]
    names = [re.sub(r"\(.*?\)", "", n).strip() for n in names]
    desc  = ", ".join(names) or "no distinct attributes"
    return (
        "USER: <image>\n"
        f"Describe this garment with: {desc}. "
        "Focus on color, material, silhouette, style, neckline, sleeves, patterns. "
        "Do not mention any person or background.\n"
        "ASSISTANT:"
    )

def build_if_prompt(row):
    cls   = (row["class_label"] or "").strip().lower()
    title = (row["product_title"] or "").strip().rstrip(".")
    if not cls and not title:
        return ""
    toks  = title.split()
    if len(toks)>10: 
        title = " ".join(toks[:10]) + "…"
    return (
        "USER: <image>\n"
        f"Describe this {cls} titled “{title}” in vivid detail. "
        "Focus on its color, fabric, pattern or embroidery, silhouette, drape, length, accents. "
        "Do not mention any person or setting.\n"
        "ASSISTANT:"
    )

def find_image_path(img_id, base):
    for ext in (".jpg",".jpeg",".png"):
        p = base/f"{img_id}{ext}"
        if p.exists(): return p
    return None

def preprocess_image(p): 
    return Image.open(p).convert("RGB").resize((336,336), Image.LANCZOS)

def clean_up():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def load_ckpt():
    if CKPT_JSON.exists():
        return json.loads(CKPT_JSON.read_text())
    return {"processed": [], "results": []}

def save_ckpt(x):
    CKPT_JSON.write_text(json.dumps(x))

indo = pd.read_csv(INDO_META_CSV, dtype=str).fillna("")
indo["id"]          = "indo_" + indo["image_id"]
indo["image_path"]  = indo["image_id"].apply(lambda x: str(find_image_path(x, INDO_IMG_DIR)))
indo["prompt"]      = indo.apply(build_if_prompt, axis=1)

fp = pd.read_csv(FP_ATTR_CSV, dtype=str)
fp["id"]            = "fp_" + fp["image_id"]
fp["image_path"]    = fp["image_id"].apply(lambda x: str(find_image_path(x, FP_IMG_DIR)))
fp["prompt"]        = fp["attributes"].apply(build_fp_prompt)


mega = pd.concat([
    fp[["id","image_path","prompt"]],
    indo[["id","image_path","prompt"]]
], ignore_index=True).set_index("id")
mega = mega[mega["prompt"].astype(bool)]  

print("total rows:", len(mega))
print("prompts: ", mega["prompt"].head(3).tolist())

total rows: 4000
prompts:  ['USER: <image>\nDescribe this garment with: gown, asymmetrical, high waist, floor, asymmetric, one shoulder, plastic. Focus on color, material, silhouette, style, neckline, sleeves, patterns. Do not mention any person or background.\nASSISTANT:', 'USER: <image>\nDescribe this garment with: symmetrical, regular, loose, normal waist, no waistline, midi, wrist-length, single breasted, zip-up. Focus on color, material, silhouette, style, neckline, sleeves, patterns. Do not mention any person or background.\nASSISTANT:', 'USER: <image>\nDescribe this garment with: symmetrical, tight, normal waist, no waistline, above-the-hip. Focus on color, material, silhouette, style, neckline, sleeves, patterns. Do not mention any person or background.\nASSISTANT:']


In [4]:
processor = AutoProcessor.from_pretrained(MODEL_ID)
model     = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if USE_FP16 else torch.float32,
    device_map="auto"
)
model.eval()

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

...

model-00001-of-00006.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): Q

In [5]:
ck  = load_ckpt()
done= set(ck["processed"])
out = ck["results"]

for start in tqdm(range(0, len(mega), BATCH_SIZE), desc="batches"):
    batch = mega.iloc[start:start+BATCH_SIZE]
    imgs, prompts, ids = [], [], []
    for idx, row in batch.iterrows():
        if idx in done: continue
        pth = row["image_path"]
        if not pth or not os.path.exists(pth):
            print(f"missing  {idx}")
            continue
        prom = row["prompt"]
        if not prom:
            continue
        imgs.append(preprocess_image(pth))
        prompts.append(prom)
        ids.append(idx)
    if not imgs:
        continue

    clean_up()
    inputs = processor(images=imgs, text=prompts,
                       return_tensors="pt", padding=True, truncation=True
                      ).to(DEVICE)
    with torch.cuda.amp.autocast(enabled=USE_FP16), torch.no_grad():
        gen = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    raws = processor.batch_decode(gen, skip_special_tokens=True)
    for uid, raw in zip(ids, raws):
        desc  = raw.split("ASSISTANT:",1)[-1].strip()
        sents = re.split(r'(?<=[.!?])\s+', desc)
        if sents and not re.search(r'[.!?]$', sents[-1]):
            sents = sents[:-1] or [sents[0]+'.']
        final= " ".join(sents).strip()
        out.append({"image_id":uid, "description": final})
        done.add(uid)

    if (start//BATCH_SIZE) % 5 == 0:
        save_ckpt({"processed":list(done),"results":out})
        pd.DataFrame(out).to_csv(OUTPUT_CSV, index=False)
        print(f"checkpoint {start//BATCH_SIZE}, total={len(out)}")

save_ckpt({"processed":list(done),"results":out})
pd.DataFrame(out).to_csv(OUTPUT_CSV, index=False)
print("done kashyap boss")

batches:   0%|          | 0/1000 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/tmp/ipykernel_31/266551105.py:27: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=USE_FP16), torch.no_grad():
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
batches:   0%|          | 1/1000 [00:17<4:59:23, 17.98s/it]

checkpoint 0, total=4


batches:   1%|          | 6/1000 [01:46<4:55:41, 17.85s/it]

checkpoint 5, total=24


batches:   1%|          | 11/1000 [03:09<4:40:46, 17.03s/it]

...

checkpoint 990, total=3961


batches: 100%|█████████▉| 996/1000 [4:38:57<01:08, 17.12s/it]

checkpoint 995, total=3981


batches: 100%|██████████| 1000/1000 [4:40:05<00:00, 16.81s/it]

done kashyap boss
